# Optimizing fantasy basketball

This notebook accompanies "Better rankings for fantasy basketball." It aims to provide additional evidence, via simulation, of the strength of the procedures defined therein. 

## Retrospective drafting

We will use data from the 2021-22 season to simulate a real draft. This allows us a fertile testing ground for various strategies, which work under the presumption that we know exact statistics for the season.

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
from itertools import combinations
from collections import Counter
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import copy
import cvxpy
from unidecode import unidecode
from itertools import combinations

from src.drafting_agents import SimpleAgent, HAgent, H2Agent, PAgent
from src.simulation import run_draft, run_multiple_seasons
from src.helper_functions import check_team_eligibility, calculate_coefficients, calculate_scores_from_coefficients,combinatorial_calculation

pd.set_option('display.max_rows', 150)
pd.options.mode.chained_assignment = None  # default='warn'

categories = ['pts', 'trb', 'ast', 'stl', 'blk', 'fg3','tov', 'fg_pct','ft_pct']

In [2]:
#this should really be in helper functions

def setup(season):
    season_str = str(season - 1) + '-' + str(season)[2:]
    stat_df = pd.read_csv('../data/stat_data/' + season_str + '_complete.csv')\
            [['PLAYER_ID','PTS','REB','AST','STL','BLK','FG3M','TO','FGM','FGA','FTM','FTA','date']]
    stat_df.columns = ['id','pts','trb','ast','stl','blk','fg3','tov','fg','fga','ft','fta','date']

    player_df = pd.read_csv('../data/player_id_reference.csv')
    position_df = pd.read_csv('../data/positions.csv')

    essential_info = stat_df.merge(player_df[['id','player']])

    position_df = position_df[position_df['season'] == season] #previously 2022
    positions = position_df.set_index(['player'])['pos']
    
    essential_info['date'] = pd.to_datetime(essential_info['date'])
    essential_info['week'] = essential_info['date'].dt.isocalendar()['week']

    weekly_df_index = pd.MultiIndex.from_product([pd.unique(essential_info['player'])
                                                 ,pd.unique(essential_info['week'])]
                                                 ,names = ['player','week'])
    weekly_df = essential_info.drop(columns = ['date','id']).groupby(['player','week']).sum()
    season_df = pd.DataFrame(weekly_df, index = weekly_df_index ).fillna(0)
    
    return season_df, positions

In [3]:
season = 2023
season_df, positions = setup(season)

Different platforms define positions in different ways. We will use the position data that we downloaded from kaggle

In [4]:
positions.head()

player
A.J. Green       {'SG'}
A.J. Lawson      {'SG'}
AJ Griffin       {'SF'}
Aaron Gordon     {'PF'}
Aaron Holiday    {'PG'}
Name: pos, dtype: object

The most important infrastructure is code simulating an actual NBA fantasy season, so that we can evaluate the success of our strategies. Actually, since our model is probabilistic, we want to run many seasons and get the average # of wins.

We'll synthesize the 2021 season into weeks of play, and randomly sample from it for the season

In [5]:
season_df.head()

pts   trb   ast  stl  blk   fg3  tov    fg   fga    ft  \
player        week                                                            
Tobias Harris 42    42.0  16.0   4.0  4.0  2.0   7.0  2.0  17.0  35.0   1.0   
              43    47.0  26.0  14.0  5.0  3.0  10.0  3.0  17.0  39.0   3.0   
              44    55.0  23.0   9.0  5.0  3.0   8.0  3.0  21.0  37.0   5.0   
              45    62.0  22.0  13.0  8.0  0.0   4.0  7.0  23.0  58.0  12.0   
              46     0.0   0.0   0.0  0.0  0.0   0.0  0.0   0.0   0.0   0.0   

                     fta  
player        week        
Tobias Harris 42     2.0  
              43     3.0  
              44     9.0  
              45    13.0  
              46     0.0

We've imported a function that simulates multiple seasons, based on the dataframe with statistics from the 2021 season

In [6]:
print(run_multiple_seasons.__doc__)

Simulate multiple seasons with the same drafters 
    
    Weekly performances are sampled from a dataframe of real season performance
    Teams win weeks by winning more categories than their opponents. They win seasons by winning the
    most weeks of all players 
    
    Args:
        teams: player assignment dict, as produced by the run_draft() functoin
        season_df: dataframe of weekly numbers per players. These will be sampled to simulate seasons
        n_seasons: number of seasons to simulate
        n_weeks: number of weeks per season
        winner_take_all: If True, the winner of a majority of categories in a week gets a point.
                         If false, each player gets a point for each category won 
        return_cat_results: If True, return detailed results on category wins 
        
    Returns:
        Series of winning percentages with the structure
         team_number : winning_fraction  
    


We also imported a function that simulates the drafting process, with agents that pick players based on rules. Drafts are typically run in the snake format. We'll consider a single round to be the back and forth of the snake- so for example with three drafters, player one picks, player two picks, player three picks two players, player two picks, and player one picks

In [7]:
print(run_draft.__doc__)

Run a snake draft

    Snake drafts wrap around like 1 -> 2 -> 3 -> 3 -> 2 -> 1 -> 1 -> 2 -> 3 etc. 
    
    Args:
        agents: list of Agents, which are required to have make_pick() methods
        n_rounds: number of rounds to do of the snake draft. Each drafter will get n_rounds * 2 players
        
    Returns:
        dictionary of player assignments with the structure
         {'player name' : team_number } 
    


When we test a strategy, we want to see if it works broadly, not just in any one position. So we will create a helper function that tries a strategy in each position against some default agent in all the other positions

In [8]:
def try_strategy(primary_agent
                 , default_agent
                 , season_df
                 , n_seasons
                 , winner_take_all = True):

    victory_res = [[] for i in range(12)]
    detailed_res = pd.DataFrame()
    for i in range(12):
        #we need to deepcopy the agents so that they don't share references with each other
        agents = [copy.deepcopy(default_agent) for x in range(i)] + \
                    [copy.deepcopy(primary_agent)] + \
                    [copy.deepcopy(default_agent) for x in range(11-i)]
        
        teams = run_draft(agents ,13)
        res, details = run_multiple_seasons(teams = teams
                                   , season_df = season_df
                                   , categories = categories
                                   , n_seasons = n_seasons
                                   , winner_take_all = winner_take_all
                                   , return_detailed_results = True)
        detailed_res = pd.concat([detailed_res, details.loc[i,:]])
        victory_res[i] = (res.get(i)) if (res.get(i)) is not None else 0
        
    return victory_res, detailed_res


def get_results_of_strategy(primary_agent
                 , default_agent
                 , season_df
                 , n_seasons
                 , winner_take_all = True):
    
    victory_res, detailed_res = try_strategy(primary_agent
                                             , default_agent
                                             , season_df
                                             , n_seasons
                                             , winner_take_all)
    return np.mean(victory_res), detailed_res


One last bit of infrastructure worth mentioning is a function to check if a team is valid, based on its players' positions. We will use it to validate whether we can add a particular player to an existing team. Surprisingly, this is not a trivial problem. 

Many players can slot into multiple positions, which makes it complicated to determine which positions are still open and which are not. The most elegant way to solve the problem is to model it as a 'phase one' optimization problem, whereby we  model the constraints with an extra variable 'z' loosening the constraints, and aim to minimize z. Fortunately we don't have to set that up manually- optimization software will do it automatically if we set up the optimization problem as is and give a dummy objective of zero. 

One might argue that our model needs to be an integer program to handle its' decision variables. This would not be ideal, since integer programs are much slower than linear programs.  Luckily we don't need to worry about that; the problem is an assignment problem and therefore the results will be integers. 

The function that checks for feasibility is a helper function used by the drafting Agents to make sure they are choosing players that fit their team. The docstring is printed below

In [9]:
print(check_team_eligibility.__doc__)

Checks if a team is eligible or not, based on the players' possible positions

    The function works by setting up an optimization problem for assigning players to team positions
    If the optimization problem is infeasible, the team is not eligible
    
    Args:
        players:Lists of players, which are themselves lists of eligible positions. E.g. 
                [['SF','PF'],['C'],['SF']]

    Returns:
        True or False, depending on if the team is found to be eligible or not

    


In [10]:
def summarize_res(res_dict, primary, default):
    
    primary_matchup = primary + ' vs ' + default
    default_matchup = default + ' vs ' + primary

    wta_primary = [(k,v[primary_matchup]['wta']['res']) for k,v in res_dict.items()]
    print(primary_matchup + ' wta')
    print(wta_primary)
    print(np.mean([x[1] for x in wta_primary]))
    
    tot_primary = [(k,v[primary_matchup]['tot']['res']) for k,v in res_dict.items()]
    print(primary_matchup + ' tot')
    print(tot_primary)
    print(np.mean([x[1] for x in tot_primary]))
          
    wta_default = [(k,v[default_matchup]['wta']['res']) for k,v in res_dict.items()]
    print(default_matchup + ' wta')
    print(wta_default)
    print(np.mean([x[1] for x in wta_default]))
          
    tot_default = [(k,v[default_matchup]['tot']['res']) for k,v in res_dict.items()]
    print(default_matchup + ' tot')
    print(tot_default)
    print(np.mean([x[1] for x in tot_default]))
          
    win_fraction = pd.concat([v[primary_matchup]['wta']['detail'].loc['win'].mean() + \
                              v[primary_matchup]['wta']['detail'].loc['tie'].mean()/2
                               for k,v in res_dict.items()], axis = 1).mean(axis = 1)
    print('win_fractions_wta')
    print(win_fraction)
    print(np.mean(win_fraction))
    
    win_fraction = pd.concat([v[primary_matchup]['tot']['detail'].loc['win'].mean() + \
                          v[primary_matchup]['tot']['detail'].loc['tie'].mean()/2
                           for k,v in res_dict.items()], axis = 1).mean(axis = 1)
    
    print('win_fractions_tot')
    print(win_fraction)
    print(np.mean(win_fraction))

    for year in [2021,2022,2023]:
        print(year)
        c_wta = res_dict[year][primary_matchup]['wta']['detail'].loc['win'].reset_index().drop(columns = ['result'])
        res_wta = combinatorial_calculation(c_wta, 1-c_wta, g_scores.columns).mean()
        print('wta')
        print('5cat')
        print(res_wta)
        print('mean')
        print(c_wta.mean().mean())

        c_tot = res_dict[year][primary_matchup]['tot']['detail'].loc['win'].reset_index().drop(columns = ['result'])
        res_tot = combinatorial_calculation(c_tot, 1-c_tot, g_scores.columns).mean()
        print('tot')
        print('5cat')
        print(res_tot)
        print('mean')
        print(c_tot.mean().mean())

# H vs G vs Z

In [11]:
seasons = [2021,2022,2023]
n_players = 12 * 13
res_dict = {}

seasons = [2022]

for season in seasons:
    
    season_df, positions = setup(season)
    
    all_players = pd.unique(season_df.index.get_level_values('player'))
    mean_of_means_1, var_of_means_1, mean_of_vars_1 = calculate_coefficients(season_df, all_players)
    first_order_scores = calculate_scores_from_coefficients(season_df
                                                       ,mean_of_means_1
                                                       ,var_of_means_1
                                                       ,mean_of_vars_1
                                                       ,alpha_weight = 1
                                                       ,beta_weight = 0)

    first_order_score_totals = first_order_scores.sum(axis = 1).sort_values(ascending = False)

    representative_player_set = first_order_score_totals.index[0:n_players]
    mean_of_means_2, var_of_means_2, mean_of_vars_2 = calculate_coefficients(season_df, representative_player_set)
    
    g_scores = calculate_scores_from_coefficients(season_df
                                                       ,mean_of_means_2
                                                       ,var_of_means_2
                                                       ,mean_of_vars_2
                                                       ,alpha_weight = 1
                                                       ,beta_weight = 1)
    
    g_order = g_scores.sum(axis = 1).sort_values(ascending = False)
    
    z_plus_scores = calculate_scores_from_coefficients(season_df
                                                       ,mean_of_means_2
                                                       ,var_of_means_2
                                                       ,mean_of_vars_2
                                                       ,alpha_weight = 1
                                                       ,beta_weight = 0)
    
    z_plus_order = z_plus_scores.sum(axis = 1).sort_values(ascending = False)
    
    g_agent = SimpleAgent(order = g_order
                           ,positions = positions)
    z_plus_agent = SimpleAgent(order = z_plus_order
                               ,positions = positions)
    
    h_agent_wta = H2Agent(season_df, positions, n_punts = 0, winner_take_all = True)
    h_agent_tot = H2Agent(season_df, positions, n_punts = 0, winner_take_all = False)

    agents = [({'wta' : h_agent_wta, 'tot' : h_agent_tot},'H'),
             ({'wta' : z_plus_agent, 'tot' : z_plus_agent},'Z+'),
             ({'wta' : g_agent, 'tot' : g_agent},'G')] 

    agent_pairs = combinations(agents, 2)
    
    agent_pairs = [(agents[0],agents[2])]
    
    seasonal_res_dict = {}
    
    print(str(season))
    
    for pair in agent_pairs:
        agent_0 = pair[0][0]
        agent_0_name = pair[0][1]
        agent_1 = pair[1][0]
        agent_1_name = pair[1][1]

        print(agent_0_name + ' against 11 ' + agent_1_name)
        res_wta, detailed_res_wta = get_results_of_strategy(agent_0['wta'], agent_1['wta'], season_df, 10000)
        res_tot, detailed_res_tot = get_results_of_strategy(agent_0['tot'], agent_1['tot'], season_df, 10000, winner_take_all = False)
        print('Winner take all: ' + str(round(res_wta*100,2)) + '%')
        print('Total categories: ' + str(round(res_tot*100,2)) + '%')
        seasonal_res_dict[agent_0_name + ' vs ' + agent_1_name] = {'wta' :
                                                                      {'res' : res_wta
                                                                       ,'detail' : detailed_res_wta  
                                                                      }
                                                                  ,'tot' :
                                                                      {'res' : res_tot
                                                                       ,'detail' : detailed_res_tot  
                                                                      }
                                                                 }
                         
        '''
        print(agent_1_name + ' against 11 ' + agent_0_name)
        res_wta, detailed_res_wta = get_results_of_strategy(agent_1['wta'], agent_0['wta'], season_df, 10000)
        res_tot, detailed_res_tot = get_results_of_strategy(agent_1['tot'], agent_0['tot'], season_df, 10000, winner_take_all = False)
        print('Winner take all: ' + str(round(res_wta*100,2)) + '%')
        print('Total categories: ' + str(round(res_tot*100,2)) + '%')
        seasonal_res_dict[agent_1_name + ' vs ' + agent_0_name] =  {'wta' :
                                                                      {'res' : res_wta
                                                                       ,'detail' : detailed_res_wta  
                                                                      }
                                                                  ,'tot' :
                                                                      {'res' : res_tot
                                                                       ,'detail' : detailed_res_tot  
                                                                      }
                                                                 }
        '''
    res_dict[season] = seasonal_res_dict

2022
H against 11 G
Iteration
0
Players
[]
Candidate
Nikola Jokic
Self means
pts    0.0
trb    0.0
ast    0.0
stl    0.0
blk    0.0
fg3    0.0
tov    0.0
ftp    0.0
fgp    0.0
dtype: float64
Rest of team means
pts   -1.348468
trb   -0.749304
ast   -1.441320
stl   -0.393197
blk   -0.214454
fg3   -0.412236
tov    1.243628
ftp   -0.607322
fgp   -0.353190
dtype: float64
Next players sorted
player
Joel Embiid           0.671557
Trae Young            0.669587
Karl-Anthony Towns    0.662953
Jayson Tatum          0.658657
LaMelo Ball           0.649116
dtype: float64
Iteration
1
Players
[]
Candidate
Nikola Jokic
Self means
pts    0.0
trb    0.0
ast    0.0
stl    0.0
blk    0.0
fg3    0.0
tov    0.0
ftp    0.0
fgp    0.0
dtype: float64
Rest of team means
pts   -1.348468
trb   -0.749304
ast   -1.441320
stl   -0.393197
blk   -0.214454
fg3   -0.412236
tov    1.243628
ftp   -0.607322
fgp   -0.353190
dtype: float64
Next players sorted
player
Joel Embiid           0.671557
Trae Young            0.669

Next players sorted
player
Mitchell Robinson    0.637803
Evan Mobley          0.633023
Jarrett Allen        0.623743
Jaylen Brown         0.618189
Herbert Jones        0.617274
dtype: float64
Iteration
0
Players
['Nikola Jokic', 'Luka Doncic', 'Donovan Mitchell', 'Jakob Poeltl', 'Christian Wood']
Candidate
Ja Morant
Self means
pts    4.407725
trb    5.483899
ast    4.869732
stl    1.092603
blk    1.809211
fg3    1.307239
tov   -5.435260
ftp   -3.401834
fgp    2.631318
dtype: float64
Rest of team means
pts   -2.508904
trb   -1.760228
ast   -2.719636
stl   -1.039333
blk   -1.070320
fg3   -0.642710
tov    2.321667
ftp   -0.867429
fgp   -0.721437
dtype: float64
Next players sorted
player
Julius Randle     0.583022
Andrew Wiggins    0.582610
Kyle Kuzma        0.579679
Bam Adebayo       0.578909
Malik Monk        0.577860
dtype: float64
Iteration
1
Players
['Nikola Jokic', 'Luka Doncic', 'Donovan Mitchell', 'Jakob Poeltl', 'Christian Wood']
Candidate
Bam Adebayo
Self means
pts    4.407725
tr

Iteration
0
Players
['Nikola Jokic', 'Luka Doncic', 'Donovan Mitchell', 'Jakob Poeltl', 'Christian Wood', 'Andrew Wiggins', 'Ja Morant', 'Russell Westbrook', 'Andre Drummond']
Candidate
De'Aaron Fox
Self means
pts    5.605083
trb    6.917917
ast    7.028401
stl    1.759421
blk    1.520673
fg3   -0.191599
tov   -8.646967
ftp   -7.932516
fgp    2.767842
dtype: float64
Rest of team means
pts   -1.536646
trb   -1.179211
ast   -1.096652
stl   -0.500364
blk   -0.957244
fg3   -0.595029
tov    1.268320
ftp   -0.245480
fgp   -0.503742
dtype: float64
Next players sorted
player
Isaiah Hartenstein    0.587793
Mason Plumlee         0.585721
Jusuf Nurkic          0.585162
Montrezl Harrell      0.585060
Hassan Whiteside      0.584663
dtype: float64
Iteration
1
Players
['Nikola Jokic', 'Luka Doncic', 'Donovan Mitchell', 'Jakob Poeltl', 'Christian Wood', 'Andrew Wiggins', 'Ja Morant', 'Russell Westbrook', 'Andre Drummond']
Candidate
De'Aaron Fox
Self means
pts    5.605083
trb    6.917917
ast    7.02840

Iteration
0
Players
['Joel Embiid']
Candidate
Luka Doncic
Self means
pts    1.817547
trb    1.781013
ast    0.387703
stl    0.230253
blk    1.302320
fg3   -0.226190
tov   -1.244687
ftp    0.581006
fgp    0.422980
dtype: float64
Rest of team means
pts   -1.785575
trb   -1.365735
ast   -2.002375
stl   -0.657668
blk   -0.789580
fg3   -0.486178
tov    1.723445
ftp   -0.815069
fgp   -0.555092
dtype: float64
Next players sorted
player
Donovan Mitchell     0.557256
Rudy Gobert          0.552158
Darius Garland       0.550744
Jonas Valanciunas    0.549935
Mikal Bridges        0.549036
dtype: float64
Iteration
1
Players
['Joel Embiid']
Candidate
Luka Doncic
Self means
pts    1.817547
trb    1.781013
ast    0.387703
stl    0.230253
blk    1.302320
fg3   -0.226190
tov   -1.244687
ftp    0.581006
fgp    0.422980
dtype: float64
Rest of team means
pts   -2.502415
trb   -0.323755
ast   -2.769289
stl   -0.749749
blk    1.005533
fg3    0.170525
tov    2.248981
ftp   -2.882796
fgp   -0.930862
dtype: floa

Next players sorted
player
Cade Cunningham             0.563952
Ja Morant                   0.557513
Tyler Herro                 0.556907
Kentavious Caldwell-Pope    0.556159
Will Barton                 0.551387
dtype: float64
Iteration
0
Players
['Joel Embiid', 'Luka Doncic', 'Jonas Valanciunas', 'Domantas Sabonis', 'Derrick White', 'Julius Randle']
Candidate
Malik Monk
Self means
pts    4.572064
trb    7.046065
ast    5.007539
stl   -0.436941
blk    1.380303
fg3   -0.527910
tov   -6.959623
ftp   -0.339694
fgp    0.204527
dtype: float64
Rest of team means
pts   -2.276689
trb   -1.565659
ast   -2.539444
stl   -1.210926
blk   -1.039126
fg3   -0.585667
tov    2.086404
ftp   -0.699997
fgp   -0.641708
dtype: float64
Next players sorted
player
Kentavious Caldwell-Pope    0.515615
Tyler Herro                 0.515230
Will Barton                 0.514291
Cade Cunningham             0.513476
Bam Adebayo                 0.510673
dtype: float64
Iteration
1
Players
['Joel Embiid', 'Luka Doncic', 

...

Iteration
0
Players
['DeMar DeRozan', 'LeBron James', 'Tyrese Maxey', 'Gary Trent Jr.', 'Shai Gilgeous-Alexander', 'Monte Morris', 'Bogdan Bogdanovic', "Devonte' Graham", 'Tyus Jones', 'Bojan Bogdanovic', 'Pat Connaughton', 'Jusuf Nurkic']
Candidate
Anfernee Simons
Self means
pts    2.576735
trb   -5.313837
ast    2.180489
stl    0.436189
blk   -5.801951
fg3    2.632407
tov    1.348506
ftp    2.864673
fgp   -1.580713
dtype: float64
Rest of team means
pts    0.0
trb    0.0
ast    0.0
stl    0.0
blk    0.0
fg3    0.0
tov    0.0
ftp    0.0
fgp    0.0
dtype: float64
Iteration
0
Players
[]
Candidate
Stephen Curry
Self means
pts    0.0
trb    0.0
ast    0.0
stl    0.0
blk    0.0
fg3    0.0
tov    0.0
ftp    0.0
fgp    0.0
dtype: float64
Rest of team means
pts   -1.348468
trb   -0.749304
ast   -1.441320
stl   -0.393197
blk   -0.214454
fg3   -0.412236
tov    1.243628
ftp   -0.607322
fgp   -0.353190
dtype: float64
Next players sorted
player
DeMar DeRozan    4.662813
Kevin Durant     4.654119
Ja

Iteration
1
Players
['DeMar DeRozan', 'James Harden', 'Jimmy Butler', 'Gary Trent Jr.']
Candidate
Keldon Johnson
Self means
pts    3.313674
trb   -0.758923
ast    3.968729
stl    2.055102
blk   -1.575261
fg3   -0.137412
tov   -2.474380
ftp    4.577215
fgp   -1.219776
dtype: float64
Rest of team means
pts   -1.189726
trb   -4.673181
ast    2.347850
stl    0.205109
blk   -5.274766
fg3    5.200657
tov    0.138844
ftp    0.115356
fgp   -6.030613
dtype: float64
Next players sorted
player
CJ McCollum                 4.554433
Shai Gilgeous-Alexander     4.550419
Kentavious Caldwell-Pope    4.549418
Jaylen Brown                4.549218
Marcus Smart                4.546944
dtype: float64
Iteration
2
Players
['DeMar DeRozan', 'James Harden', 'Jimmy Butler', 'Gary Trent Jr.']
Candidate
CJ McCollum
Self means
pts    3.313674
trb   -0.758923
ast    3.968729
stl    2.055102
blk   -1.575261
fg3   -0.137412
tov   -2.474380
ftp    4.577215
fgp   -1.219776
dtype: float64
Rest of team means
pts    -1.300

Next players sorted
player
Malik Beasley        4.627415
Tyus Jones           4.625643
Immanuel Quickley    4.623498
Reggie Jackson       4.623349
Patty Mills          4.622038
dtype: float64
Iteration
0
Players
['DeMar DeRozan', 'James Harden', 'Jimmy Butler', 'Gary Trent Jr.', 'CJ McCollum', "D'Angelo Russell", 'Bogdan Bogdanovic', 'Jae Crowder', "Devonte' Graham"]
Candidate
Malik Beasley
Self means
pts    3.025660
trb   -3.975329
ast    5.832950
stl    2.834852
blk   -4.304674
fg3    3.447121
tov   -1.731349
ftp    4.651816
fgp   -4.745385
dtype: float64
Rest of team means
pts   -1.536646
trb   -1.179211
ast   -1.096652
stl   -0.500364
blk   -0.957244
fg3   -0.595029
tov    1.268320
ftp   -0.245480
fgp   -0.503742
dtype: float64
Next players sorted
player
Patty Mills          4.475430
Reggie Jackson       4.472323
Immanuel Quickley    4.460604
Luke Kennard         4.459471
Tyus Jones           4.457697
dtype: float64
Iteration
1
Players
['DeMar DeRozan', 'James Harden', 'Jimmy Butle

player
Khris Middleton    4.580063
Gary Trent Jr.     4.580051
Tobias Harris      4.567883
Jordan Poole       4.563889
Mo Bamba           4.559889
dtype: float64
Iteration
0
Players
['DeMar DeRozan', 'James Harden', 'Jimmy Butler', 'Saddiq Bey']
Candidate
CJ McCollum
Self means
pts    3.380082
trb    0.357646
ast    4.590093
stl    1.096990
blk   -1.645446
fg3   -0.112659
tov   -2.766769
ftp    4.463970
fgp   -1.551657
dtype: float64
Rest of team means
pts   -2.455827
trb   -1.773169
ast   -2.714657
stl   -1.128451
blk   -1.087866
fg3   -0.578051
tov    2.283611
ftp   -0.933302
fgp   -0.772758
dtype: float64
Next players sorted
player
Jaylen Brown               4.435727
Shai Gilgeous-Alexander    4.432315
Christian Wood             4.430229
Keldon Johnson             4.429634
D'Angelo Russell           4.424339
dtype: float64
Iteration
1
Players
['DeMar DeRozan', 'James Harden', 'Jimmy Butler', 'Saddiq Bey']
Candidate
Keldon Johnson
Self means
pts    3.380082
trb    0.357646
ast    4.5

Iteration
1
Players
['DeMar DeRozan', 'James Harden', 'Jimmy Butler', 'Saddiq Bey', 'Marcus Smart', 'Keldon Johnson', 'Bogdan Bogdanovic', 'Jordan Clarkson']
Candidate
Devonte' Graham
Self means
pts    3.728119
trb   -1.156078
ast    4.972470
stl    2.076661
blk   -4.250085
fg3    2.262379
tov   -2.584230
ftp    4.600390
fgp   -3.690136
dtype: float64
Rest of team means
pts   -0.591311
trb   -2.582352
ast    0.798601
stl   -0.223597
blk   -4.538517
fg3    4.018678
tov    0.621940
ftp    0.632300
fgp   -4.657938
dtype: float64
Next players sorted
player
Malik Beasley        4.587406
Reggie Jackson       4.582625
Immanuel Quickley    4.579916
Tyus Jones           4.574986
Patty Mills          4.574738
dtype: float64
Iteration
2
Players
['DeMar DeRozan', 'James Harden', 'Jimmy Butler', 'Saddiq Bey', 'Marcus Smart', 'Keldon Johnson', 'Bogdan Bogdanovic', 'Jordan Clarkson']
Candidate
Devonte' Graham
Self means
pts    3.728119
trb   -1.156078
ast    4.972470
stl    2.076661
blk   -4.250085
f

# Punting

In [21]:
seasons = [2021,2022,2023]
n_players = 12 * 13
res_dict_punt = {}

for season in seasons:
    
    season_df, positions = setup(season)
    
    all_players = pd.unique(season_df.index.get_level_values('player'))
    mean_of_means_1, var_of_means_1, mean_of_vars_1 = calculate_coefficients(season_df, all_players)
    first_order_scores = calculate_scores_from_coefficients(season_df
                                                       ,mean_of_means_1
                                                       ,var_of_means_1
                                                       ,mean_of_vars_1
                                                       ,alpha_weight = 1
                                                       ,beta_weight = 0)

    first_order_score_totals = first_order_scores.sum(axis = 1).sort_values(ascending = False)

    representative_player_set = first_order_score_totals.index[0:n_players]
    mean_of_means_2, var_of_means_2, mean_of_vars_2 = calculate_coefficients(season_df, representative_player_set)
    
    g_scores = calculate_scores_from_coefficients(season_df
                                                   ,mean_of_means_2
                                                   ,var_of_means_2
                                                   ,mean_of_vars_2
                                                   ,alpha_weight = 1
                                                   ,beta_weight = 1).fillna(0)

    g_agent_punt = PAgent(scores = g_scores
                           ,positions = positions
                            ,n_punts = 4)

    z_plus_scores = calculate_scores_from_coefficients(season_df
                                                       ,mean_of_means_2
                                                       ,var_of_means_2
                                                       ,mean_of_vars_2
                                                       ,alpha_weight = 1
                                                       ,beta_weight = 0).fillna(0)
    
    z_agent_punt = PAgent(scores = z_plus_scores
                   ,positions = positions
                    ,n_punts = 4)
    
    z_agent_no_punt = PAgent(scores = z_plus_scores
               ,positions = positions
                ,n_punts = 0)

    h_agent_wta = HAgent(season_df, positions, n_punts = 4, winner_take_all = True)
    h_agent_tot = HAgent(season_df, positions, n_punts = 4, winner_take_all = False)

    agent_pairs = [(({'wta' : h_agent_wta, 'tot' : h_agent_tot},'H-p')
                    ,({'wta' : z_agent_no_punt, 'tot' : z_agent_no_punt},'Z+'))
                  ,(({'wta' : g_agent_punt, 'tot' : g_agent_punt},'G-p')
                     ,({'wta' : z_agent_no_punt, 'tot' : z_agent_no_punt},'Z+'))]
    
    seasonal_res_dict = {}
    
    print(str(season))
    
    for pair in agent_pairs:
        agent_0 = pair[0][0]
        agent_0_name = pair[0][1]
        agent_1 = pair[1][0]
        agent_1_name = pair[1][1]

        print(agent_0_name + ' against 11 ' + agent_1_name)
        res_wta, detailed_res_wta = get_results_of_strategy(agent_0['wta'], agent_1['wta'], season_df, 1000)
        res_tot, detailed_res_tot = get_results_of_strategy(agent_0['tot'], agent_1['tot'], season_df, 1000, winner_take_all = False)
        print('Winner take all: ' + str(round(res_wta*100,2)) + '%')
        print('Total categories: ' + str(round(res_tot*100,2)) + '%')
        seasonal_res_dict[agent_0_name + ' vs ' + agent_1_name] = {'wta' :
                                                                      {'res' : res_wta
                                                                       ,'detail' : detailed_res_wta  
                                                                      }
                                                                  ,'tot' :
                                                                      {'res' : res_tot
                                                                       ,'detail' : detailed_res_tot  
                                                                      }
                                                                 }
                         

        print(agent_1_name + ' against 11 ' + agent_0_name)
        res_wta, detailed_res_wta = get_results_of_strategy(agent_1['wta'], agent_0['wta'], season_df, 1000)
        res_tot, detailed_res_tot = get_results_of_strategy(agent_1['tot'], agent_0['tot'], season_df, 1000, winner_take_all = False)
        print('Winner take all: ' + str(round(res_wta*100,2)) + '%')
        print('Total categories: ' + str(round(res_tot*100,2)) + '%')
        seasonal_res_dict[agent_1_name + ' vs ' + agent_0_name] =  {'wta' :
                                                                      {'res' : res_wta
                                                                       ,'detail' : detailed_res_wta  
                                                                      }
                                                                  ,'tot' :
                                                                      {'res' : res_tot
                                                                       ,'detail' : detailed_res_tot  
                                                                      }
                                                                 }
    
    res_dict_punt[season] = seasonal_res_dict

2021
H-p against 11 Z+


KeyboardInterrupt: 